In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import contextily as ctx
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb

df = pd.read_parquet("final_model_data.parquet")

# show all columns
pd.set_option('display.max_columns', None)
df


In [ ]:
# fix a few row issues

pd.set_option('display.max_columns', None)
df.columns = df.columns.str.replace(r"[()',]", "", regex=True).str.strip()
df = df.replace({True: 1, False: 0})

df.rename(columns={"fore": "fare"}, inplace=True)
df.rename(columns={"class": "service"}, inplace=True)
df

In [ ]:
# starting to develop the idea for the target

df["profit_per_hour"] = df["fare"] / (df["durationsec"] / 3600).round(2)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.loc[df["profit_per_hour"] > 1]

In [ ]:
# remove outliers for more robust model

df["profit_per_hour"].std() * 3

In [ ]:
df = df[df["profit_per_hour"] < 397.95]
df

In [ ]:
# create clusters with coordinates and airport data (experimental feature)

pu_features = ["PUx", "PUy", "airport"]
do_features = ["DOx", "DOy", "airport"]

scaler_pu = StandardScaler()
pu_scaled = scaler_pu.fit_transform(df[pu_features])

scaler_do = StandardScaler()
do_scaled = scaler_do.fit_transform(df[do_features])


In [ ]:
# find elbow for location and airport encoded clusters

inertia = []
K_range = range(7, 20) 

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(pu_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(K_range, inertia)
plt.show()

In [ ]:
# SET K HERE

pu_k = 9
kmeans = KMeans(n_clusters=pu_k, random_state=42, n_init=10)
df["cluster"] = kmeans.fit_predict(pu_scaled)


In [ ]:
sns.scatterplot(x=df["PUx"], y=df["PUy"], hue=df["cluster"], palette="tab10")
plt.show()

In [ ]:

df_taxi = df[(df["service"] == 0) & (df["fare"] > 0)]
df_uber = df[(df["service"] == 1) & (df["fare"] > 0)]
df_lyft = df[(df["service"] == 2) & (df["fare"] > 0)]


In [ ]:
df_uber.head(20)

In [ ]:
# turn the clusters and service into booleans for training

df = pd.get_dummies(df, columns=["cluster"])
df = pd.get_dummies(df, columns=["service"])
df

In [ ]:
# drop taxis, from here on my models use uber and lyft data only

df = df[df["service_0"] == 0]
df["service_1"].value_counts()

In [ ]:
# convert to a geopandas df

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["PUx"], df["PUy"]), crs="EPSG:2263")  # Adjust EPSG if needed


In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
kde = sns.kdeplot(
    x=gdf.geometry.x, 
    y=gdf.geometry.y, 
    ax=ax, 
    fill=True, 
    cmap="viridis",
    alpha=0.7
)

# add basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=gdf.crs)

# EPSG instead of actual long/lat
ax.set_title("NYC Driver Earnings Heatmap ($/hr)")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# fix colorbar issue
fig.colorbar(ax.collections[-1], ax=ax, label="Earnings per Hour ($)")

plt.show()



In [ ]:
# don't rewrite over final parquet, commented out on purpose

#df.to_parquet("final_model_data.parquet", engine="pyarrow")

In [ ]:
# full feature modeling (minus temp and PU_Bronx)

cluster_features = [col for col in df.columns if col.startswith("cluster_")]

features = [
    
    "second_of_day",
    "day_of_year",
    "morning_rush",
    "evening rush",
    "temp",
    "holiday",
    "weekend",
    "airport",
    "congestion",
    "PUx",
    "PUy",
    "DOx",
    "DOy",
    "PU_Brooklyn",
    "PU_Manhattan",
    "PU_Queens",
    "PU_Staten Island",
    "service_1",
] + cluster_features


In [ ]:
# train duration model with full features


df = df.dropna()

y = df["durationsec"] 
X = df[features]

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
}

random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
y_pred_xgb = random_search.best_estimator_.predict(X_test)

xgb_duration = random_search.best_estimator_


In [ ]:
xgb_duration = random_search.best_estimator_

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb:.2f}")
r2_via_metric = r2_score(y_test, y_pred_xgb)
print("R² via r2_score:", r2_via_metric)

In [ ]:
# assess for feature selection

feature_importances = xgb_duration.feature_importances_

importance_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": feature_importances.round(2)
})

importance_df

In [ ]:
# refit model with features greater than 0.01

features2 = importance_df[importance_df["Importance"] > 0.01]["Feature"].tolist()
features2


In [ ]:
# train duration model with reduced features

y = df["durationsec"] 
X = df[features2]

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

param_dist = {
    'n_estimators': [300, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [9, 13, 15],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
}

random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
y_pred_xgb = random_search.best_estimator_.predict(X_test)

xgb_duration = random_search.best_estimator_


xgb_duration = random_search.best_estimator_

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb:.2f}")
r2_via_metric = r2_score(y_test, y_pred_xgb)
print("R² via r2_score:", r2_via_metric)


In [ ]:
# train duration model with original features from other notebook

features = ["second_of_day", "day_of_year", "PUx", "PUy", "DOx", "DOy", "distance", "morning_rush", "evening rush",
            "prcp", "temp", "holiday", "weekend", "airport", "congestion", "PU_Bronx", "PU_Brooklyn",
            "PU_Manhattan", "PU_Queens", "PU_Staten Island", "DO_Bronx", "DO_Brooklyn", 
            "DO_Manhattan", "DO_Queens", "DO_Staten Island"]

y = df["durationsec"] 
X = df[features]

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

param_dist = {
    'n_estimators': [300, 500],
    'learning_rate': [0.01, 0.02],
    'max_depth': [9, 13],
    'subsample': [0.8],
    'colsample_bytree': [0.4, 0.6],
}

random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
y_pred_xgb = random_search.best_estimator_.predict(X_test)

xgb_duration2 = random_search.best_estimator_


mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb:.2f}")
r2_via_metric = r2_score(y_test, y_pred_xgb)
print("R² via r2_score:", r2_via_metric)

Best Duration Model of .86 R² and about three minutes off (mean absolute error)

In [ ]:
# train earnings model with full features

X = df[features]
y = df["profit_per_hour"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)


In [ ]:

xgb_model = xgb.XGBRegressor(random_state=3)


params = {
    "n_estimators": [300, 500],
    "max_depth": [13, 15],
    "learning_rate": [0.02, 0.05, 0.1],
    "subsample": [0.6, 0.8],
    "colsample_bytree": [0.4, 0.6]
}

random_search = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=params,
        n_iter=10,  
        cv=3,  
        n_jobs=-1, 
        random_state=3
    )

random_search.fit(X_train, y_train)


In [ ]:
xgb_model = random_search.best_estimator_

y_pred = xgb_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"R² score: {r2}")
print(f"Best parameters are: {random_search.best_params_}")
print(f"These predictions are off by about ${mae:.2f}")"


In [ ]:
# dummy regressor for baseline #1

# Create a naive baseline model that always predicts the mean of y_train
dummy_model = DummyRegressor(strategy="mean")
dummy_model.fit(X_train, y_train)

# Predict using the naive baseline model
y_pred_dummy = dummy_model.predict(X_test)

# Evaluate the naive model
mae_dummy = mean_absolute_error(y_test, y_pred_dummy)
r2_dummy = r2_score(y_test, y_pred_dummy)

# Evaluate the trained XGBoost model
y_pred_xgb = xgb_model.predict(X_test)  # Ensure you are using the best trained model
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Print comparison results
print(f"📊 Naive Baseline MAE: {mae_dummy:.2f}, R²: {r2_dummy:.4f}")
print(f"🚀 XGBoost Model MAE: {mae_xgb:.2f}, R²: {r2_xgb:.4f}")

# Determine if the model beats the baseline
if mae_xgb < mae_dummy:
    print("✅ The XGBoost model performs better than the naive baseline!")
else:
    print("❌ The XGBoost model does NOT outperform the naive baseline.")


In [ ]:
y_pred.mean()

In [ ]:
# checking feature importances

feature_importances = xgb_model.feature_importances_

importance_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": feature_importances.round(2)
})

importance_df

In [ ]:
# refit model with features greater than 0.01

features2 = importance_df[importance_df["Importance"] > 0.01]["Feature"].tolist()
features2 = df[features2].drop(columns="distance")
features2.columns


In [ ]:
# train the simpler model, zeroing in on params

# Prepare data
X = df[features2]
y = df["profit_per_hour"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

params = {
    "n_estimators": [800],
    "max_depth": [15],
    "learning_rate": [0.02],
    "subsample": [0.8],
    "colsample_bytree": [0.6],
    "reg_alpha": [0.05, 0.1] 
}

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=3)

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=params,
    n_iter=8,  
    cv=3,  
    n_jobs=12, 
    random_state=3
)

random_search.fit(X_train, y_train)

# get best model
xgb_model = random_search.best_estimator_

y_pred = xgb_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"R² score: {r2:.4f}")
print(f"Best parameters are: {random_search.best_params_}")
print(f"These predictions are off by about ${mae:.2f}")


In [ ]:
# whoops, so I won't save this model. distance left in - heavy leakage...

features2

In [ ]:
# don't save over best models unless intentional

"""
import pickle

with open("duration_model.pkl", "wb") as f:
    pickle.dump(xgb_duration2, f)

# Save the earnings per hour model
with open("earnings_model.pkl", "wb") as f:
    pickle.dump(xgb_model, f)
"""

So far best model performance is R² of .29 and MAE of $17.13

In [ ]:
# trying an MLP regressor

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

nn_model = MLPRegressor(
    hidden_layer_sizes=(32, 16),
    activation="relu",
    solver="adam",
    max_iter=100,
    early_stopping=True,
    verbose=True,
    random_state=3
)

nn_model.fit(X_train_scaled, y_train)
y_pred_nn = nn_model.predict(X_test_scaled)

mae_nn = mean_absolute_error(y_test, y_pred_nn)
print(f"🧠 Neural Network MAE (with Scaler): ${mae_nn:.2f}")


In [ ]:
df[df["temp"] == df["temp"].max()]

In [ ]:
df